# 解析モデル
![](img/2d-mur/model.png)

# 各種パラメータ

In [13]:
%mkdir result
%rm result/*.png
%mkdir img/2d-mur

import numpy as np

# 解析領域の分割数
nx = 120
ny = 120 

nstep = 2000 # 計算ステップ総数

# セルサイズ
dx = 0.005
dy = 0.005

# 電界
ex = np.zeros((nx, ny))
ey = np.zeros((nx, ny))
ez = np.zeros((nx, ny))

# 磁界
hx = np.zeros((nx, ny))
hy = np.zeros((nx, ny))
hz = np.zeros((nx, ny))

# 係数
aex  = np.zeros((nx, ny))
aey  = np.zeros((nx, ny))
aez  = np.zeros((nx, ny))
bexy = np.zeros((nx, ny))
beyx = np.zeros((nx, ny))
bezx = np.zeros((nx, ny))
bezy = np.zeros((nx, ny))
amx  = np.zeros((nx, ny))
amy  = np.zeros((nx, ny))
amz  = np.zeros((nx, ny))
bmxy = np.zeros((nx, ny))
bmyx = np.zeros((nx, ny))
bmzx = np.zeros((nx, ny))
bmzy = np.zeros((nx, ny))

# 媒質定数
epsbk  = 1 # 背景媒質
mubk   = 1
sigebk = 0
sigmbk = 0
epsd  = np.full((nx+1, ny+1), epsbk ) # 比誘電率
sgmed = np.full((nx+1, ny+1), sigebk) # 導電率
mud   = np.full((nx+1, ny+1), mubk  ) # 比透磁率
sgmmd = np.full((nx+1, ny+1), sigmbk) # 磁気伝導率

# 誘電体
ic   = int(nx/2) # 中心
jc   = int(ny/2)
lx2  = 20        # 長さ
ly2  = 20
epsr = 3         # 比誘電率

# 電流源
duration = 0.1e-9         # パルス幅
t0       = 4.0 * duration # ピーク時刻
ifed = ic - lx2 - 20      # 給電位置
jfed = jc

# 自由空間
eps0 = 8.854188e-12
mu0  = 1.256637e-6
c    = 2.9979246e8



mkdir: cannot create directory ‘result’: File exists
rm: cannot remove 'result/*.png': No such file or directory
mkdir: cannot create directory ‘img/2d-mur’: File exists


# 初期化

In [17]:
t = 0
v  = c / np.sqrt(epsbk*mubk)
dt = 0.99999 / ( v * np.sqrt(1/(dx**2) + 1/(dy**2)) )

# 誘電体中の媒質定数
for j in range(jc - ly2, jc + ly2):
    for i in range(ic - lx2, ic + lx2):
        epsd[i][j]  = epsr
        mud[i][j]   = 1
        sgmed[i][j] = 0
        sgmmd[i][j] = 0

epsz  = (epsd[ifed][jfed] + epsd[ifed-1][jfed] + epsd[ifed][jfed-1] + epsd[ifed-1][jfed-1])/4 * eps0
befed = dt / epsz # 電流源の係数 

for j in range(ny):
    for i in range(nx):
        epsx = (epsd[i][j]  + epsd[i][j-1] )/2 * eps0
        sgex = (sgmed[i][j] + sgmed[i][j-1])/2    
        a = sgex /2 *dt / epsx
        aex[i][j]  = (1 - a)/(1 + a)
        beyx[i][j] = dt / epsx / (1 + a) /dy

        epsy = (epsd[i][j]  + epsd[i-1][j] )/2 * eps0
        sgey = (sgmed[i][j] + sgmed[i-1][j])/2
        a = sgey /2 *dt /epsy
        aey[i][j]  = (1 - a)/(1 + a)
        bexy[i][j] = dt / epsy / (1 + a) / dx

        epsz = (epsd[i][j]  + epsd[i-1][j]  + epsd[i][j-1]  + epsd[i-1][j-1] )/4 * eps0
        sgez = (sgmed[i][j] + sgmed[i-1][j] + sgmed[i][j-1] + sgmed[i-1][j-1])/4
        a = sgez /2 *dt /epsy
        aez[i][j]  = (1 - a)/(1 + a)
        bezy[i][j] = dt / epsz / (1 + a) / dy
        bezx[i][j] = dt / epsz / (1 + a) / dx

        mux = (mud[i][j]    + mud[i-1][j]  )/2 * mu0
        sgmx = (sgmmd[i][j] + sgmmd[i-1][j])/2
        a = sgmx /2 *dt / mux
        amx[i][j]  = (1 - a)/(1 + a)
        bmxy[i][j] = dt / mux / (1 + a) / dy

        muy = (mud[i][j] + mud[i][j-1])/2 * mu0
        sgmy = (sgmmd[i][j] + sgmmd[i][j-1])/2
        a = sgmy /2 *dt /muy
        amy[i][j]  = (1 - a)/(1 + a)
        bmyx[i][j] = dt / muy / (1 + a) / dx

        muz = mud[i][j] * mu0
        sgmz = sgmmd[i][j]
        a = sgmz /2 *dt /muz
        amz[i][j]  = (1 - a)/(1 + a)
        bmzx[i][j] = dt / muz / (1 + a) / dx
        bmzy[i][j] = dt / muz / (1 + a) / dy
    



# 電流源

In [18]:
def feed():
    iz = np.exp(-((t - 0.5*dt - t0)/duration)**2)
    ez[ifed][jfed] -= befed * iz/(dx*dy)

# 電磁界の計算

In [20]:
def e_cal():
    # Ex
    for j in range(1, ny-1):
        for i in range(0, nx-1):
            ex[i][j] = aex[i][j] * ex[i][j] + bexy[i][j] * (hz[i][j] - hz[i][j-1])
    
    # Ey
    for j in range(0, ny-1):
        for i in range(1, nx-1):
            ey[i][j] = aey[i][j] * ey[i][j] + beyx[i][j] * (hz[i][j] - hz[i-1][j])

    # Ez
    for j in range(1, ny-1):
        for i in range(1, nx-1):
            ez[i][j] = aez[i][j] * ez[i][j] + bezx[i][j] * (hy[i][j] - hy[i-1][j]) - bezy[i][j] * (hx[i][j] - hx[i][j-1])

def h_cal():
    # Hx
    for j in range(1, ny-1):
        for i in range(0, nx-1):
            hx[i][j] = amx[i][j] * hx[i][j] + bmxy[i][j] * (ez[i][j+1] - ez[i][j])
    
    # Hy
    for j in range(0, ny-1):
        for i in range(1, nx-1):
            hy[i][j] = amy[i][j] * hy[i][j] + bmyx[i][j] * (ez[i+1][j] - ez[i][j])

    # Hz
    for j in range(1, ny-1):
        for i in range(1, nx-1):
            hz[i][j] = amz[i][j] * hz[i][j] - bmzx[i][j] * (ey[i+1][j] - ey[i][j]) + bmzy[i][j] * (ex[i][j+1] - ex[i][j])



# 吸収境界条件